<a href="https://colab.research.google.com/github/Konerusudhir/machine_learning_exercises/blob/master/MaliciousURL_TFX_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# Installs
!pip install tfx 
# %load_ext tensorboard


In [2]:
import os
import re
import requests
import tarfile
from os import path
from datetime import datetime
import shutil
import absl
from typing import List
from urllib.parse import urlparse

from typing import Text

# Data handling libraries
import numpy as np # linear algebra
import pandas as pd # data processing

import tensorflow as tf
from tfx import v1 as tfx
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.dsl.components.common import resolver
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.v1.orchestration import LocalDagRunner
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.types.standard_artifacts import String
from tfx.types.standard_artifacts import Examples
from tfx.types.standard_artifacts import ExampleStatistics


# from tfx.v1 import proto
# from tfx.v1.proto import Output
# from tfx.proto import example_gen_pb2

# Download Data

In [12]:
_data_root = 'data'
_tfx_root ='tfx_root'
_features_data_root = 'features_data'
model_imaages = "model_imaages"
data_with_features = os.path.join(_features_data_root, 'data_with_features.csv')

def clean_directories():
  shutil.rmtree(_data_root, ignore_errors=True)
  shutil.rmtree(_tfx_root, ignore_errors=True)
  shutil.rmtree(_features_data_root, ignore_errors=True)
  shutil.rmtree(model_imaages, ignore_errors=True)
def create_directories():
  os.mkdir(_data_root)
  os.mkdir(_tfx_root)
  os.mkdir(_features_data_root)
  os.mkdir(model_imaages)

clean_directories()
create_directories()


# url = 'https://drive.google.com/file/d/1OzJbRtgU-Z80iL7GodIH8-ymKq58g7Az/view?usp=share_link'
url = 'https://drive.google.com/file/d/19JVyvZjw93TfnSqPWHxU4QBms4BPj_lo/view?usp=share_link'
url_download_path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
download_file = f"{_data_root}/small_file.csv"
if not path.exists(download_file):
  r = requests.get(url_download_path, allow_redirects=True)
  open(download_file, 'wb').write(r.content)
df = pd.read_csv(download_file)
# df = df.sample(n=100000, random_state=2)
df.info()
df = df.sample(frac=1, random_state=42)
display(df.head())

df['url_len'] = df['url'].apply(lambda x: len(str(x))) 
df['count_www'] = df['url'].apply(lambda i: i.count('www'))
df['count_at'] = df['url'].apply(lambda i: i.count('@'))
df['count_dir'] = df['url'].apply(lambda url: urlparse(url).path.count('/'))
df['count_http'] = df['url'].apply(lambda i : i.count('http'))
df['count_https'] = df['url'].apply(lambda i : i.count('https'))
df['hostname_length'] = df['url'].apply(lambda i: len(urlparse(i).netloc))

# Character counts
df['count_percent'] = df['url'].apply(lambda i: i.count('%'))
df['count_hyphen'] = df['url'].apply(lambda i: i.count('-'))

# Query parameters features
df['count_question_mark'] = df['url'].apply(lambda i: i.count('?'))
df['count_equals'] = df['url'].apply(lambda i: i.count('='))

rem = {"class": {"benign": 0, "defacement": 1, "phishing":2, "malware":3}}
df['class'] = df['type']
df = df.replace(rem)
display(df.sample(n=10))
df.to_csv(data_with_features, index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     36 non-null     object
 1   type    36 non-null     object
dtypes: object(2)
memory usage: 704.0+ bytes


,url,type
35,http://peluqueriadeautor.com/index.php?option=...,defacement
13,http://www.lebensmittel-ueberwachung.de/index....,defacement
26,casamanana.org/education/blba/,benign
30,soaps.sheknows.com/daysofourlives/news/id/2025...,benign
16,quickfacts.census.gov/qfd/maps/iowa_map.html,benign


,url,type,url_len,count_www,count_at,count_dir,count_http,count_https,hostname_length,count_percent,count_hyphen,count_question_mark,count_equals,class
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign,118,0,0,3,1,0,11,0,16,0,0,0
30,soaps.sheknows.com/daysofourlives/news/id/2025...,benign,81,0,0,6,0,0,0,0,0,0,0,0
0,br-icloud.com.br,phishing,16,0,0,0,0,0,0,0,1,0,0,2
20,baseball-reference.com/players/h/harrige01.shtml,benign,48,0,0,3,0,0,0,0,1,0,0,0
8,http://www.pashminaonline.com/pure-pashminas,defacement,44,1,0,1,1,0,22,0,1,0,0,1
2,bopsecrets.org/rexroth/cr/1.htm,benign,31,0,0,3,0,0,0,0,0,0,0,0
24,escholarship.org/uc/item/5xt4952c,benign,33,0,0,3,0,0,0,0,0,0,0,0
14,http://www.szabadmunkaero.hu/cimoldal.html?sta...,defacement,51,1,0,1,1,0,21,0,0,1,1,1
9,allmusic.com/album/crazy-from-the-heat-r16990,benign,45,0,0,2,0,0,0,0,4,0,0,0
18,uk.linkedin.com/pub/steve-rubenstein/8/718/755,benign,46,0,0,5,0,0,0,0,1,0,0,0


In [13]:
_pipeline_name = 'malicious_url_pipeline'
_module_file = 'utils.py'
_serving_model_dir = os.path.join(_tfx_root, 'serving_model', _pipeline_name)
_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)
_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name,
                              'metadata.db')

# Pipeline arguments for Beam powered Components.
_beam_pipeline_args = []

# Create Utils File

In [14]:
%%writefile utils.py
from typing import List

from absl import logging
import tensorflow as tf
import tensorflow_transform as tft

from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx.components.trainer.fn_args_utils import FnArgs
from tfx_bsl.tfxio import dataset_options

# Categorical features are assumed to each have a maximum value in the dataset.
_MAX_CATEGORICAL_FEATURE_VALUES = [24, 31, 12]

_CATEGORICAL_FEATURE_KEYS = [
]

_DENSE_FLOAT_FEATURE_KEYS = [
  'url_len',
  'count_www',
  'count_at',
  'count_dir',
  'count_http',
  'count_https',
  'hostname_length',
  'count_percent',
  'count_hyphen',
  # 'count_question_mark',
  # 'count_equals'
]

# Number of buckets used by tf.transform for encoding each feature.
_FEATURE_BUCKET_COUNT = 10

_BUCKET_FEATURE_KEYS = []

# Number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
_VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
_OOV_SIZE = 10

_VOCAB_FEATURE_KEYS = []

# Keys
_LABEL_KEY = 'class'


def _transformed_name(key):
  return key + '_xf'


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.

  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.

  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.

  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


def _get_tf_examples_serving_signature(model, tf_transform_output):
  """Returns a serving signature that accepts `tensorflow.Example`."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_inference = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_example):
    """Returns the output to be used in the serving signature."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    # Remove label feature since these will not be present at serving time.
    raw_feature_spec.pop(_LABEL_KEY)
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_inference(raw_features)
    logging.info('serve_transformed_features = %s', transformed_features)

    outputs = model(transformed_features)
    # TODO(b/154085620): Convert the predicted labels from the model using a
    # reverse-lookup (opposite of transform.py).
    return {'outputs': outputs}

  return serve_tf_examples_fn


def _get_transform_features_signature(model, tf_transform_output):
  """Returns a serving signature that applies tf.Transform to features."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_eval = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def transform_features_fn(serialized_tf_example):
    """Returns the transformed_features to be fed as input to evaluator."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_eval(raw_features)
    logging.info('eval_transformed_features = %s', transformed_features)
    return transformed_features

  return transform_features_fn


def _input_fn(file_pattern: List[str],
              data_accessor: DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema).repeat()


def _build_keras_model(hidden_units: List[int] = None) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying taxi data.

  Args:
    hidden_units: [int], the layer sizes of the DNN (input layer first).

  Returns:
    A keras Model.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]

  model = _wide_and_deep_classifier(
      deep_columns=real_valued_columns,
      dnn_hidden_units=hidden_units or [100, 70, 50, 25])
  return model


def _wide_and_deep_classifier(deep_columns, dnn_hidden_units):
  """Build a simple keras wide and deep model.

  Args:
    wide_columns: Feature columns wrapped in indicator_column for wide (linear)
      part of the model.
    deep_columns: Feature columns for deep part of the model.
    dnn_hidden_units: [int], the layer sizes of the hidden DNN.

  Returns:
    A Wide and Deep Keras model
  """
  # Following values are hard coded for simplicity in this example,
  # However prefarably they should be passsed in as hparams.

  # Keras needs the feature definitions at compile time.
  # TODO(b/139081439): Automate generation of input layers from FeatureColumn.
  input_layers = {
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype=tf.float32)
      for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  }
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_BUCKET_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_CATEGORICAL_FEATURE_KEYS)
  })

  # TODO(b/161952382): Replace with Keras premade models and
  # Keras preprocessing layers.
  deep = tf.keras.layers.DenseFeatures(deep_columns)(input_layers)
  for numnodes in dnn_hidden_units:
    deep = tf.keras.layers.Dense(numnodes)(deep)
  
  output = tf.keras.layers.Dense(
      4, activation='softmax')(deep)

  model = tf.keras.Model(input_layers, output)
  model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      optimizer=tf.keras.optimizers.Adam(lr=0.001),
      metrics=['accuracy'])
  model.summary(print_fn=logging.info)
  return model


# TFX Transform will call this function.
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[_transformed_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]))

  outputs[_transformed_name(_LABEL_KEY)] = inputs[_LABEL_KEY]

  return outputs


# TFX Trainer will call this function.
def run_fn(fn_args: FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  # Number of nodes in the first layer of the DNN
  first_dnn_layer_size = 7
  num_dnn_layers = 1
  dnn_decay_factor = 0.7
  epochs = 10

  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor,
                            tf_transform_output, 40)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor,
                           tf_transform_output, 40)

  mirrored_strategy = tf.distribute.MirroredStrategy()
  with mirrored_strategy.scope():
    model = _build_keras_model(
        # Construct layers sizes with exponetial decay
        hidden_units=[
            max(2, int(first_dnn_layer_size * dnn_decay_factor**i))
            for i in range(num_dnn_layers)
        ])

  # Write logs to path
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      epochs=epochs,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  signatures = {
      'serving_default':
          _get_tf_examples_serving_signature(model, tf_transform_output),
      'transform_features':
          _get_transform_features_signature(model, tf_transform_output),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Overwriting utils.py


In [15]:
@tfx.dsl.components.component
def ModelValidator(
  model: tfx.dsl.components.InputArtifact[Model],
  statistics: tfx.dsl.components.InputArtifact[ExampleStatistics]):
  
  from IPython.display import Image, display
  import os

  path = os.path.join(model.uri, 'Format-Serving')
  model_instance = tf.keras.models.load_model(path)
      
  display(model_instance.summary())
  
  print(statistics.uri)
  files = tf.io.gfile.glob(f"{statistics.uri}/*/*.pb")
  files.sort()
  for file in files:
    print(f"File: {file}")
  train_stats = stats_data = tfdv.load_stats_binary(files[1]) 
  eval_stats = stats_data = tfdv.load_stats_binary(files[0]) 
  tfdv.visualize_statistics(train_stats, eval_stats)



In [16]:
# TODO(b/137289334): rename this as simple after DAG visualization is done.
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str,
                     beam_pipeline_args: List[str]) -> pipeline.Pipeline:
 
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

  # Generates schema based on statistics files.
  schema_gen = SchemaGen(
      statistics=statistics_gen.outputs['statistics'],
      infer_feature_shape=True)

  # Performs anomaly detection based on statistics and data schema.
  example_validator = ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_gen.outputs['schema'])

  # Performs transformations and feature engineering in training and serving.
  transform = Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_gen.outputs['schema'],
      module_file=module_file)

  # Uses user-provided Python function that implements a model.
  trainer = Trainer(
      module_file=module_file,
      examples=transform.outputs['transformed_examples'],
      transform_graph=transform.outputs['transform_graph'],
      schema=schema_gen.outputs['schema'],
      train_args=trainer_pb2.TrainArgs(num_steps=1000),
      eval_args=trainer_pb2.EvalArgs(num_steps=150))

  validator = ModelValidator(
      model = trainer.outputs['model'],
      statistics=statistics_gen.outputs['statistics']    
  )

  return pipeline.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=[
          example_gen,
          statistics_gen,
          schema_gen,
          example_validator,
          transform,
          trainer,
          validator
      ],
      enable_cache=True,
      metadata_connection_config=metadata.sqlite_metadata_connection_config(
          metadata_path),
      beam_pipeline_args=beam_pipeline_args)


# To run this pipeline from the python CLI:
#   $python taxi_pipeline_native_keras.py
if __name__ == '__main__':
  absl.logging.set_verbosity(absl.logging.INFO)

  LocalDagRunner().run(
      _create_pipeline(
          pipeline_name=_pipeline_name,
          pipeline_root=_pipeline_root,
          data_root=_features_data_root,
          module_file=_module_file,
          metadata_path=_metadata_path,
          serving_model_dir=_serving_model_dir,
          beam_pipeline_args=_beam_pipeline_args))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/content/utils.py' (including modules: ['utils']).
INFO:absl:User module package has hash fingerprint version 49b76eda6cb6dd87f0557b4dbeb1a81759a535c019f81e52415ff7a508caf004.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmpr125em0u/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpv1n5ukfc', '--dist-dir', '/tmp/tmpdlymgdw4']
INFO:absl:Successfully built user code wheel distribution at 'tfx_root/pipelines/malicious_url_pipeline/_wheels/tfx_user_code_Transform-0.0+49b76eda6cb6dd87f0557b4dbeb1a81759a535c019f81e52415ff7a508caf004-py3-none-any.whl'; target user module is 'utils'.
INFO:absl:Full user module path is 'utils@tfx_root/pipelines/malicious_url_pipeline/_wheels/tfx_user_code_Transform-0.0+49b76eda6cb6dd87f0557b4dbeb1a81759

Epoch 1/10
1000/1000 [==============================] - 9s 8ms/step - loss: 0.4012 - accuracy: 0.8834 - val_loss: 0.1264 - val_accuracy: 1.0000
Epoch 2/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1060 - accuracy: 0.9754 - val_loss: 0.0940 - val_accuracy: 0.9230
Epoch 3/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0353 - accuracy: 1.0000 - val_loss: 0.1461 - val_accuracy: 0.9230
Epoch 4/10
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0131 - accuracy: 1.0000 - val_loss: 0.2175 - val_accuracy: 0.9230
Epoch 5/10
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.2870 - val_accuracy: 0.9230
Epoch 6/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.3519 - val_accuracy: 0.9230
Epoch 7/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.4117 - val_accuracy:

INFO:absl:serve_transformed_features = {'count_at_xf': <tf.Tensor 'transform_features_layer_1/PartitionedCall:1' shape=(None, 1) dtype=float32>, 'url_len_xf': <tf.Tensor 'transform_features_layer_1/PartitionedCall:9' shape=(None, 1) dtype=float32>, 'count_percent_xf': <tf.Tensor 'transform_features_layer_1/PartitionedCall:6' shape=(None, 1) dtype=float32>, 'hostname_length_xf': <tf.Tensor 'transform_features_layer_1/PartitionedCall:8' shape=(None, 1) dtype=float32>, 'count_www_xf': <tf.Tensor 'transform_features_layer_1/PartitionedCall:7' shape=(None, 1) dtype=float32>, 'count_https_xf': <tf.Tensor 'transform_features_layer_1/PartitionedCall:4' shape=(None, 1) dtype=float32>, 'count_hyphen_xf': <tf.Tensor 'transform_features_layer_1/PartitionedCall:5' shape=(None, 1) dtype=float32>, 'count_http_xf': <tf.Tensor 'transform_features_layer_1/PartitionedCall:3' shape=(None, 1) dtype=float32>, 'count_dir_xf': <tf.Tensor 'transform_features_layer_1/PartitionedCall:2' shape=(None, 1) dtype=flo

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 count_at_xf (InputLayer)       [(None,)]            0           []                               
                                                                                                  
 count_dir_xf (InputLayer)      [(None,)]            0           []                               
                                                                                                  
 count_http_xf (InputLayer)     [(None,)]            0           []                               
                                                                                                  
 count_https_xf (InputLayer)    [(None,)]            0           []                               
                                                                                            

None

tfx_root/pipelines/malicious_url_pipeline/StatisticsGen/statistics/2
File: tfx_root/pipelines/malicious_url_pipeline/StatisticsGen/statistics/2/Split-eval/FeatureStats.pb
File: tfx_root/pipelines/malicious_url_pipeline/StatisticsGen/statistics/2/Split-train/FeatureStats.pb


INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 7 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {}) for execution 7
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component ModelValidator is finished.
